# 🌿 AgroBot - Advanced AI Crop Assistant

Features
- **Multilingual NLP**: English + Hindi + Hinglish support
- **95%+ Accuracy CNN**: MobileNetV2 Transfer Learning
- **Professional Green Theme UI**: Gradio Blocks interface
- **PlantVillage Dataset**: 38 Plant Disease Classes

---

In [1]:

!pip install -q gradio tensorflow tensorflow-datasets pillow transformers sentencepiece

In [2]:


import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import gradio as gr
import numpy as np
from PIL import Image
import difflib
import re


try:
    from transformers import pipeline
    print("✅ Transformers library imported!")
except:
    print("⚠️ Transformers not available")

print("✅ All libraries imported successfully!")

✅ Transformers library imported!
✅ All libraries imported successfully!


In [3]:

print("Starting AgroBot setup...")

try:
    from google.colab import files
    model = tf.keras.models.load_model('/content/plant_disease_model.h5')
    print("✅ Loaded existing model!")
except:
    print("⚠️ No saved model found. Using your existing notebook's model.")
    print("Let's continue with the multilingual NLP and UI upgrade!")

print("✅ Setup complete! Now adding remaining components...")

Starting AgroBot setup...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


✅ Loaded existing model!
✅ Setup complete! Now adding remaining components...


In [4]:
!pip install gradio tensorflow pillow


In [5]:
import difflib

agri_database = {
    "potato": {
        "yellowing of leaves": "Cause: Nutrient imbalance. Treatment: Apply phosphate-rich fertilizer.",
        "curling of leaves": "Cause: Water stress. Treatment: Improve drainage.",
        "blemish in stem": "Cause: Early blight. Treat with copper-based fungicide."
    },
    "tomato": {
        "yellow spots": "Cause: Mosaic virus. Remove infected leaves.",
        "white fungus": "Cause: Powdery mildew. Use sulfur-based spray.",
        "wilting": "Cause: Root rot. Improve drainage."
    },
    "rice": {
        "brown spots": "Cause: Leaf blast. Use tricyclazole fungicide.",
        "yellowing": "Cause: Nitrogen deficiency. Apply urea.",
        "dry tips": "Cause: Potassium deficiency. Use potash fertilizer."
    },
    "wheat": {
        "rust": "Cause: Wheat rust fungus. Use mancozeb.",
        "yellowing": "Cause: Nitrogen deficiency. Apply urea.",
        "white powder": "Cause: Powdery mildew. Use sulfur dust."
    }
}


def preprocess(text):
    text = text.lower()
    return text



def detect_crop(text):
    crops = list(agri_database.keys())


    for crop in crops:
        if crop in text:
            return crop


    best = difflib.get_close_matches(text, crops, n=1, cutoff=0.4)
    return best[0] if best else None


def detect_symptom(crop, text):
    symptoms = list(agri_database[crop].keys())


    for s in symptoms:
        if s in text:
            return s


    best = difflib.get_close_matches(text, symptoms, n=1, cutoff=0.4)
    return best[0] if best else None



def process_text(user_input):
    text = preprocess(user_input)

    crop = detect_crop(text)
    if not crop:
        return "❌ Crop not detected. Please include the crop name (e.g., potato, tomato)."

    symptom = detect_symptom(crop, text)
    if not symptom:
        return f"⚠ No matching symptom found for {crop}."

    return f"""
### 🌱 Crop: {crop.capitalize()}
### 🔍 Symptom: {symptom}
### 💡 Advice:
{agri_database[crop][symptom]}
"""


In [6]:
def process_text(user_input):
    text, words = preprocess(user_input)
    crop = detect_crop(text)

    if not crop:
        return "❌ No crop detected."

    symptom = detect_symptom(crop, text)

    if not symptom:
        return f"⚠ Symptom not found for {crop}."

    return f"""
### 🌱 Crop: {crop.capitalize()}

### 🔍 Symptom: {symptom}

### 💡 Advice:
{agri_database[crop][symptom]}
"""


In [7]:
!pip install tensorflow_hub


In [8]:
!pip install huggingface_hub


In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds


(ds_train, ds_val, ds_test), ds_info = tfds.load(
    "plant_village",
    split=["train[:80%]", "train[80%:90%]", "train[90%:]"],
    with_info=True,
    as_supervised=True
)

class_names = ds_info.features["label"].names
NUM_CLASSES = len(class_names)

IMG_SIZE = 128
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img, label

ds_train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_val   = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)
ds_test  = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [10]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 38)             │         4,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,309,542 (12.62 MB)

 Trainable params: 3,309,542 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)


Epoch 1/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 73s 49ms/step - accuracy: 0.5469 - loss: 1.6602 - val_accuracy: 0.7604 - val_loss: 0.7417
Epoch 2/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.8610 - loss: 0.4401 - val_accuracy: 0.8842 - val_loss: 0.3746
Epoch 3/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.9165 - loss: 0.2544 - val_accuracy: 0.8864 - val_loss: 0.3918
Epoch 4/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - accuracy: 0.9456 - loss: 0.1675 - val_accuracy: 0.9039 - val_loss: 0.3282
Epoch 5/5
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 36s 26ms/step - accuracy: 0.9593 - loss: 0.1202 - val_accuracy: 0.9135 - val_loss: 0.3078


In [12]:
import os

model_dir = '/content/drive/MyDrive/AgroBot_Models/'
os.makedirs(model_dir, exist_ok=True)

model_path = os.path.join(model_dir, 'plant_disease_model.h5')
model.save(model_path)

print(f"Model saved successfully to: {model_path}")
print(f"Model size: {os.path.getsize(model_path) / (1024*1024):.2f} MB")

Model saved successfully to: /content/drive/MyDrive/AgroBot_Models/plant_disease_model.h5
Model size: 37.92 MB


In [13]:
import numpy as np

def predict_image(image):
    img = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    img = tf.expand_dims(img, 0)

    preds = model.predict(img)[0]
    idx = np.argmax(preds)
    confidence = preds[idx]

    return f"### 🌿 CNN Prediction\nDisease: **{class_names[idx]}**\nConfidence: **{confidence*100:.2f}%**"


In [14]:
def agrobot(text_input, image_input):
    output = ""

    if text_input:
        output += process_text(text_input) + "\n\n"

    if image_input is not None:
        output += predict_image(image_input)

    return output


In [15]:
!find /content -name "*.h5"


/content/plant_disease_model.h5
/content/drive/MyDrive/AgroBot_Models/plant_disease_model.h5


In [ ]:

from __future__ import annotations
from typing import Iterable
import re
import difflib
import gradio as gr
from gradio.themes.base import Base
from gradio.themes.utils import colors, fonts, sizes
import tensorflow as tf
import numpy as np


class Seafoam(Base):
    def __init__(
        self,
        *,
        primary_hue: colors.Color | str = colors.emerald,
        secondary_hue: colors.Color | str = colors.blue,
        neutral_hue: colors.Color | str = colors.blue,
        spacing_size: sizes.Size | str = sizes.spacing_md,
        radius_size: sizes.Size | str = sizes.radius_md,
        text_size: sizes.Size | str = sizes.text_lg,
        font: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("Quicksand"),
            "ui-sans-serif",
            "sans-serif",
        ),
        font_mono: fonts.Font | str | Iterable[fonts.Font | str] = (
            fonts.GoogleFont("IBM Plex Mono"),
            "ui-monospace",
            "monospace",
        ),
    ):
        super().__init__(
            primary_hue=primary_hue,
            secondary_hue=secondary_hue,
            neutral_hue=neutral_hue,
            spacing_size=spacing_size,
            radius_size=radius_size,
            text_size=text_size,
            font=font,
            font_mono=font_mono,
        )
        super().set(
            body_background_fill="linear-gradient(135deg, #2E7D32, #1B5E20)",
            button_primary_background_fill="linear-gradient(90deg, *primary_300, *secondary_400)",
            button_primary_text_color="white",
            block_shadow="*shadow_drop_lg",
            block_border_width="3px",
        )

seafoam = Seafoam()


hinglish_map = {
    "aloo": "potato", "aaloo": "potato", "आलू": "potato",
    "tamatar": "tomato", "टमाटर": "tomato",
    "chawal": "rice", "चावल": "rice",
    "gehun": "wheat", "गेहूं": "wheat",
    "patte": "leaves", "पत्ते": "leaves",
    "peele": "yellow", "पीले": "yellow",
    "daag": "spots", "दाग": "spots",
    "hai": "is", "h": "is"
}

def normalize_hinglish(text):
    text = text.lower()
    for k, v in hinglish_map.items():
        text = text.replace(k, v)
    return text



agri_database = {
    "potato": {
        "yellowing of leaves": "Cause: Nutrient imbalance. Use phosphate fertilizer.",
        "curling of leaves": "Cause: Water stress. Improve drainage.",
        "blemish in stem": "Cause: Early blight. Apply copper fungicide."
    },

    "tomato": {
        "yellow spots": "Cause: Mosaic virus. Remove infected leaves.",
        "white fungus": "Cause: Powdery mildew. Use sulfur spray.",
        "wilting": "Cause: Root rot. Improve drainage."
    },

    "rice": {
        "brown spots": "Cause: Leaf blast. Use tricyclazole.",
        "yellowing": "Cause: Nitrogen deficiency. Apply urea.",
        "dry tips": "Cause: Potassium deficiency. Use potash fertilizer."
    },

    "wheat": {
        "rust": "Cause: Rust fungus. Use mancozeb.",
        "yellowing": "Cause: Nitrogen deficiency. Apply urea.",
        "white powder": "Cause: Powdery mildew. Use sulfur dust."
    }
}


def preprocess(text):
    text = text.lower()
    return text, text.split()

def detect_crop(text):
    for crop in agri_database.keys():
        if crop in text:
            return crop
        if difflib.get_close_matches(crop, text.split(), cutoff=0.6):
            return crop
    return None

def detect_symptom(crop, text):
    symptoms = agri_database[crop].keys()
    for symptom in symptoms:
        words = symptom.split()
        match_count = sum(1 for w in words if difflib.get_close_matches(w, text.split(), cutoff=0.6))
        if match_count >= len(words) // 2:
            return symptom
    return None

def process_text_multilingual(user_input):
    if not user_input.strip():
        return "❌ Please describe crop problem."

    normalized = normalize_hinglish(user_input)
    text, _ = preprocess(normalized)

    crop = detect_crop(text)
    if not crop:
        return "❌ No crop detected."

    symptom = detect_symptom(crop, text)
    if not symptom:
        return f"⚠ Symptom not found for **{crop.capitalize()}**."

    treatment = agri_database[crop][symptom]

    return f"""
### 📝 NLP Result

🌱 **Crop:** {crop.capitalize()}

🔍 **Symptom:** {symptom}

💡 **Treatment**: {treatment}
"""


def predict_image(img):
    img_processed = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img_processed = img_processed / 255.0
    img_processed = tf.expand_dims(img_processed, 0)

    preds = model.predict(img_processed)[0]
    idx = np.argmax(preds)

    return f"""
### 🌿 CNN Prediction
**Disease:** {class_names[idx]}\n
**Confidence:** {(preds[idx]*100):.2f}%
"""


css = """
textarea, input[type="text"] {
    color: white !important;
    background-color: #1B5E20 !important;
    border: 2px solid #66BB6A !important;
}
label {
    color: white !important;
    font-weight: 600;
}
.output-markdown {
    color: white !important;
}
.gradio-container {
    background: linear-gradient(135deg, #2E7D32, #1B5E20) !important;
    color: white !important;
}
button {
    background: #66BB6A !important;
    color: black !important;
    font-weight: 600 !important;
}
"""

with gr.Blocks(css=css, theme=seafoam) as demo:

    gr.Markdown("# 🌿 AgroBot - Multilingual AI Crop Assistant")

    with gr.Row():
        with gr.Column():
            text_inp = gr.Textbox(label="Describe Crop Issue (Eng/Hindi/Hinglish)", lines=3)
        with gr.Column():
            img_inp = gr.Image(label="Upload Leaf Image", sources=["upload"])

    output = gr.Markdown("### Results will appear here...")

    submit_btn = gr.Button("🔎 Analyze")
    clear_btn = gr.Button("🗑 Clear")

    def agrobot(text, img):
        result = ""
        if text:
            result += process_text_multilingual(text)
        if img is not None:
            result += predict_image(img)
        return result or "⚠ Please provide input."

    submit_btn.click(agrobot, [text_inp, img_inp], output)
    clear_btn.click(lambda: ("", None, ""), None, [text_inp, img_inp, output])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2bad4fd5f78aefb5c5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1133, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
